## Landsat Thermal Sharpening Example - Interactive

In [1]:
import ee
import folium

import openet.sharpen

# ee.Authenticate()
ee.Initialize(use_cloud_api=True)

In [2]:
viridis_palette = ['#440154', '#433982', '#30678D', '#218F8B', '#36B677', '#8ED542', '#FDE725']

image_size = 768


In [3]:
landsat_id = 'LC08_042034_20180705'
landsat_sr_img = ee.Image('LANDSAT/LC08/C01/T1_SR/{}'.format(landsat_id))

map_xy = [-120, 37]
# map_xy = landsat_sr_img.geometry().centroid().getInfo()['coordinates']
# print(map_xy)

max_zoom = 12


In [4]:
# "Prep" the Landsat SR image
# Copied from PTJPL Image.from_landsat_c1_sr()
# The "SATELLITE" property must be set on the image for the algorithm to work
input_bands = ee.Dictionary({
    'LANDSAT_5': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'pixel_qa'],
    'LANDSAT_7': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B6', 'pixel_qa'],
    'LANDSAT_8': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'pixel_qa']})
output_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'tir', 'pixel_qa']
spacecraft_id = ee.String(landsat_sr_img.get('SATELLITE'))
prep_img = landsat_sr_img \
    .select(input_bands.get(spacecraft_id), output_bands) \
    .multiply([0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.1, 1]) \
    .set({'system:index': landsat_sr_img.get('system:index'),
          'system:time_start': landsat_sr_img.get('system:time_start'),
          'system:id': landsat_sr_img.get('system:id'), 
          'SATELLITE': spacecraft_id,
         })

# Run the sharpening algorithm
output_img = openet.sharpen.thermal.landsat(prep_img)


In [5]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr="Map Data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
        show=show
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [7]:
my_map = folium.Map(location=[map_xy[1], map_xy[0]], zoom_start=max_zoom, height=image_size)

# Add the layers to the map
my_map.add_ee_layer(
    landsat_sr_img.select([3, 2, 1]), {'min': 0, 'max': 3000}, 'True Color', False)
my_map.add_ee_layer(
    landsat_sr_img.select(['B10']), 
    {'min': 2950, 'max': 3250, 'palette': viridis_palette}, 'Thermal Raw', False)
my_map.add_ee_layer(
    prep_img.select(['tir']), 
    {'min': 295, 'max': 325, 'palette': viridis_palette}, 'Thermal Original', False)
my_map.add_ee_layer(
    output_img.select(['tir_sharpened']), 
    {'min': 295, 'max': 325, 'palette': viridis_palette}, 'Thermal Sharpened')
my_map.add_ee_layer(
    prep_img.select(['tir']).subtract(output_img.select(['tir_sharpened'])), 
    {'min': -5, 'max': 5, 'palette': ['ff0000', 'ffffff', '0000ff']}, 'Difference')

my_map.add_child(folium.LayerControl())
display(my_map)